In [16]:
import pandas as pd
import numpy as np

In [17]:
data = pd.read_csv("2020-12-31-DynamicallyGeneratedHateDataset-entries-v0.1.csv")

In [18]:
data.head()

,Unnamed: 0,id,text,label,type,model_wrong,db.model_preds,status,round,split,annotator
0,1,10000,It should be legal for a husband to rape his wife,hate,notgiven,True,0.97457,dynabench entry,1,train,gnzuctwed3
1,2,10001,dalits are lowlife scum,hate,notgiven,False,0.08233,dynabench entry,1,test,cagnluiznm
2,3,10002,dalits are lowlives,hate,notgiven,True,0.92319,dynabench entry,1,train,cagnluiznm
3,4,10003,It was a better world when women dare not ques...,hate,notgiven,True,0.99006,dynabench entry,1,test,gnzuctwed3
4,5,10004,I don't work this hard so that those immigrant...,hate,notgiven,True,0.98836,dynabench entry,1,train,cagnluiznm


In [19]:
data = data[['text', 'label']]

In [20]:
data.head()

,text,label
0,It should be legal for a husband to rape his wife,hate
1,dalits are lowlife scum,hate
2,dalits are lowlives,hate
3,It was a better world when women dare not ques...,hate
4,I don't work this hard so that those immigrant...,hate


In [21]:
data.isnull().sum()

text     0
label    0
dtype: int64

In [22]:
data.label.value_counts()

hate       22124
nothate    18499
Name: label, dtype: int64

In [23]:
data.shape

(40623, 2)

In [24]:
import re
# converting the text column into lower case
data['text'] = data['text'].apply(lambda x: x.lower())
# removing special chars
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
#
data.head()

,text,label
0,it should be legal for a husband to rape his wife,hate
1,dalits are lowlife scum,hate
2,dalits are lowlives,hate
3,it was a better world when women dare not ques...,hate
4,i dont work this hard so that those immigrants...,hate


In [25]:
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.utils import resample
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix,classification_report
import re

In [26]:
# Now tokenizing the text column    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)
X[:2]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 

In [27]:
#Hereby I declare the train and test dataset.
Y = pd.get_dummies(data['label']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(32498, 336) (32498, 2)
(8125, 336) (8125, 2)


In [28]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 336, 128)          256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 336, 128)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
batch_size = 128
model.fit(X_train, Y_train, epochs = 15, batch_size=batch_size, verbose = 1)

Epoch 1/15
254/254 [==============================] - 919s 4s/step - loss: 0.6128 - accuracy: 0.6483
Epoch 2/15
254/254 [==============================] - 1016s 4s/step - loss: 0.7437 - accuracy: 0.6090
Epoch 3/15
254/254 [==============================] - 1111s 4s/step - loss: 0.5929 - accuracy: 0.6745
Epoch 4/15
254/254 [==============================] - 1137s 4s/step - loss: 0.5576 - accuracy: 0.7056
Epoch 5/15
254/254 [==============================] - 1271s 5s/step - loss: 0.5364 - accuracy: 0.7289
Epoch 6/15
254/254 [==============================] - 1235s 5s/step - loss: 0.5198 - accuracy: 0.7412
Epoch 7/15
254/254 [==============================] - 1198s 5s/step - loss: 0.5097 - accuracy: 0.7474
Epoch 8/15
254/254 [==============================] - 1227s 5s/step - loss: 0.5022 - accuracy: 0.7510
Epoch 9/15
254/254 [==============================] - 1187s 5s/step - loss: 0.4920 - accuracy: 0.7579
Epoch 10/15
254/254 [==============================] - 1270s 5s/step - loss: 0.4881

In [30]:
from tensorflow.keras.models import Model

In [31]:
final_model= Model.save(model,"LSTM.h5")